In [1]:
import scanpy as sc
import pyro

import sys
sys.path.append("../")

/opt/homebrew/anaconda3/envs/echidna/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Example 4: Echidna & Pyro

In this tutorial, we show you how to use Echidna more flexibly by accessing the underlying data in the model. This will be useful if you have prior familiarity with Pyro and aim to perform more custom work.

In [3]:
import echidna as ec
ec.pl.activate_plot_settings()

%load_ext autoreload
%autoreload 2

As in the previous example, we load our AnnData from our previous run. Keep in mind that the copy number data is now saved in `.obs`.

In [4]:
adata = sc.read_h5ad("data/F01_echidna.h5")

Loading a model reads in a pickled model object and loads the Pyro parameter store. We can see that this is true by looking at the keys in the global parameter store.

In [5]:
echidna = ec.tl.load_model(adata)

In [6]:
pyro.get_param_store().keys()

dict_keys(['eta_mean', 'c_shape', 'scale_shape', 'scale_rate', 'corr_loc', 'corr_scale'])

If you make changes to the model object or param store, you can save those changes with the following function:

In [ ]:
ec.tl.save_model(echidna, adata, overwrite=False)

The `echidna` object will contain the configuartion, the model and guide functions to run a forward pass through the model, as well as Torch tensors for the ground truth $\eta$, $c$ and $\Sigma$ from training.

In [7]:
eta, c, cov = echidna.eta_ground_truth, echidna.c_ground_truth, echidna.cov_ground_truth

Let's also see how we build the data tensors to do a forward pass. Echidna does this under-the-hood for you for safety purposes, but it may be helpful to do have access to the actual tensors. 

In [8]:
data = ec.tl.build_torch_tensors(adata, echidna.config)

A pass of the data through the model will return the X and W tensors passed through the model, after a full sampling of the model.

In [9]:
echidna.model(*data)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


(tensor([[0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0.1413, 1.0000, 1.0000,  ..., 3.0000, 3.0000, 3.0000]))

In [10]:
echidna.guide(*data)

This function uses poutine to trace the guide with the data, replay the model with that guide, and finally return nodes from the trace of the replay with the data. See for example a sampled $\eta$ compared to the ground truth of many averaged samples.

In [11]:
learned_params = ec.tl.get_learned_params(echidna, data)

In [12]:
learned_params["eta"]["value"], eta

(tensor([[-0.7405, -0.9719, -0.0742,  ...,  0.2652,  2.7870,  0.5386],
         [-1.0875, -0.5485, -1.0295,  ...,  0.3954,  0.3113,  1.5511],
         [-0.0624, -0.1475, -0.2463,  ..., -0.3997,  0.6911,  1.1359],
         ...,
         [ 1.8684,  2.3558,  2.1578,  ...,  3.3801,  2.0947,  2.6765],
         [ 2.2723,  1.7335,  1.1958,  ...,  2.1342,  1.5169,  0.5055],
         [ 2.0608,  0.9850,  1.6464,  ...,  2.3175,  0.7653,  2.6031]],
        grad_fn=<AddBackward0>),
 tensor([[0.3285, 0.3108, 0.7245,  ..., 1.9894, 2.6318, 2.2516],
         [0.2480, 0.3777, 0.6065,  ..., 2.5768, 1.9483, 1.6197],
         [0.2685, 0.4418, 0.6654,  ..., 2.4282, 1.9220, 1.3426],
         ...,
         [0.7181, 1.6986, 0.8640,  ..., 2.3416, 1.3837, 1.7297],
         [2.5549, 1.5685, 0.8891,  ..., 1.3954, 1.2125, 1.7350],
         [0.5228, 1.6219, 1.0226,  ..., 2.5157, 0.7609, 2.4471]],
        requires_grad=True))

The rest of the problem, as they say, is behind the keyboard.